<a href="https://colab.research.google.com/github/stphnhng/482Project2/blob/master/Embedding_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

#!mkdir encoder
#!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
#!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

In [95]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import json
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from nltk.tokenize import sent_tokenize

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [98]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=89978bd10d6f8ca0995941c771919c90fe63402b332e07b936adf5a8228a2e91
  Stored in directory: /tmp/pip-ephem-wheel-cache-yejferr4/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load() 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_json(file):
  with open(file, 'r') as fp:
    return json.loads(fp.read())

In [0]:
# Read in the data
data = read_json("/content/drive/My Drive/data_wInfo.json")
qa = read_json("/content/drive/My Drive/cp_alexa_qa.json")
generated_qa = read_json("/content/drive/My Drive/generated_qa.json")

In [0]:
stop_words = stopwords.words('english')

In [0]:
def parse_content(data):
  sentences = []
  outputs = []
  for section, content in data.items():
    if content['content'] != '':
      sent_toke = sent_tokenize(content['content'])
      for index, sent in enumerate(sent_toke):
        if sent == 'The W.K.':
          sent_toke[index+1] = sent + sent_toke[index+1]
        elif sent == 'Alumni include Abel Maldonado, Former California Lt.':
          sent_toke[index+1] = sent + sent_toke[index+1]
        else:
          word_list = nltk.word_tokenize(sent)
          output = [w for w in word_list if not w in stop_words]
          outputs.append(' '.join(output))
          sentences.append(sent)
  return sentences, outputs

In [0]:
sentences, outputs = parse_content(data)

In [0]:
for question in generated_qa:
    answer = generated_qa[question]
    if answer not in sentences:
        sentences.append(answer)

In [0]:
sentences_embeddings = embed(sentences)

In [0]:
outputs_embeddings = embed(outputs)

In [0]:
def cos_sim(A, B):
  return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [0]:
def match_generated(query):
    qa_generated_qs = list(generated_qa.items())
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_generated_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    print("returning match_generated:", most_similar_ans)
    print("\tgenerated_sim:", most_similar_num)
    return most_similar_ans

In [0]:
def match_qa(query):
    qa_infobox_qs = list(qa.items())[:68] # >68 are infobox questions
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_infobox_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    print("returning match_qa:", most_similar_ans)
    print("\tqa sim:", most_similar_num)
    return most_similar_ans

In [0]:
def match_infobox(query):
    qa_infobox_qs = list(qa.items())[68:] # >68 are infobox questions
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_infobox_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    print("returning match_infobox:", most_similar_ans)
    print("\tinfobox sim:", most_similar_num)
    return most_similar_ans

In [0]:
def find_sent_index(query, sentences_embeddings):
  query_embedding = embed([query])
  max_sim_index = 0
  max_sim = -1
  sim_list = []
  for index, embedding in enumerate(sentences_embeddings):
    current_sim = cos_sim(query_embedding, embedding)[0]
    sim_list.append((index, current_sim))
    if current_sim > max_sim:
      max_sim = current_sim
      max_sim_index = index
  sim_list.sort(key=lambda x: x[1], reverse=True)
#   if max_sim < 0.5:
  info_q = match_infobox(query)
  generated_q = match_generated(query)
  qa_q = match_qa(query)
  print("embedding sim:",max_sim)
  return max_sim_index, sim_list

In [197]:
index, scores = find_sent_index("?", sentences_embeddings)
if index == -1:
    print(scores)
else:
    print("embedding:", sentences[index])

returning match_infobox: 1,659 (Fall 2018)
	infobox sim: 0.8270330682096495
returning match_generated: Cal Poly received one of the largest gifts towards public education to be received in California on May 3,2017.
	generated_sim: 0.8483933707183999
returning match_qa: In 1960, control of Cal Poly San Luis Obispo and all other state colleges was transferred from the State Board of Education to an independent Board of Trustees, which later became the California State University system.
	qa sim: 0.8454570191571539
embedding sim: 0.47287977
embedding: The school is typically referred to as "Cal Poly", or simply "Poly".
